In [ ]:
# !pip install import-ipynb

#access drive
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial

x_path= "/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/clean_data/ohc3.csv"
y_path= "/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial/clean_data/depVar"



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial


In [ ]:
%run Splitting_data.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial # Change path to yours'
/content/drive/MyDrive/GRAD-C24_Machine_Learning/MLProject_KenyaFinancial


In [ ]:
X_train, X_val, y_train, y_val = training_set(x_path, y_path)

In [27]:
import xgboost
from matplotlib import pyplot as plt

#le = preprocessing.LabelEncoder()
#y_vald = le.fit_transform(y_vald.astype(str))

y_val = y_val.convert_dtypes(convert_string=True)

#eval_set=[(X_train, y_train), (X_val, y_val)]
eval_set=[(X_val, y_val)]


xgb_reg = xgboost.XGBClassifier(n_estimators=3000,
                               max_depth=12,
                               subsample=1,
                               max_bin=500, # Maximum number of discrete bins to bucket continuous features. Increasing this number improves the optimality of splits at the cost of higher computation time.
                               #objective="reg:squarederror", # for iterative learning
                               eval_metric="accuracy", # for early-stopping evaluation using eval_set
                               early_stopping_rounds=10,
                               max_delta_step=0, # Maximum delta step we allow each leaf output to be. If the value is set to 0, it means there is no constraint. If it is set to a positive value, it can help making the update step more conservative. Usually this parameter is not needed, but it might help in logistic regression when class is extremely imbalanced. Set it to value of 1-10 might help control the update.
                               verbosity=1,
                               n_jobs=-1,
                              )
# see more parameters in https://xgboost.readthedocs.io/en/stable/parameter.html#parameters-for-tree-booster

xgb_reg.fit(X_train, y_train, 
            eval_set=eval_set, 
            #verbose=False
           )

results = xgb_reg.evals_result()
lowest_score = round(min(results["validation_1"]["rmse"]), 2)

print(f"Optimal number of trees: {xgb_reg.best_ntree_limit}\n RMSE: {lowest_score}")

# plot graph
plt.figure(figsize=(10,7))
plt.plot(results["validation_0"]["rmse"], label="Training loss")
plt.plot(results["validation_1"]["rmse"], label="Validation loss")
plt.axvline(xgb_reg.best_ntree_limit, color="gray", label="Optimal tree number")
plt.xlabel("Number of trees")
plt.ylabel("Loss")
plt.legend()

TypeError: ignored

In [29]:
X_val

,Unnamed: 0,trx_yr_mo_2012_08,trx_yr_mo_2012_09,trx_yr_mo_2012_10,trx_yr_mo_2012_11,trx_yr_mo_2012_12,trx_yr_mo_2013_01,trx_yr_mo_2013_02,trx_yr_mo_2013_03,trx_yr_mo_2013_04,...,dem_i_age_yrs,dem_i_age_workage,dem_i_hhead,dem_i_children_tot,dem_i_children_inhh,edu_i_enrolled,edu_i_attain_yrs,first_trx_date_acc_f,last_trx_date_acc_f,trx_date_f
260211,260211,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.345075e+09,1.388448e+09,1.353024e+09
228583,228583,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.345075e+09,1.388448e+09,1.361750e+09
441908,441908,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.345075e+09,1.388448e+09,1.373846e+09
449153,449153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,1.0,0.0,2.0,0.0,0.0,10.0,1.365293e+09,1.380758e+09,1.372896e+09
95175,95175,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.345075e+09,1.388448e+09,1.350864e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610602,610602,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,58.0,1.0,1.0,0.0,6.0,0.0,2.0,1.354579e+09,1.382486e+09,1.355443e+09
726866,726866,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,49.0,1.0,0.0,5.0,1.0,0.0,12.0,1.352160e+09,1.380067e+09,1.368749e+09
669774,669774,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,79.0,0.0,1.0,1.0,6.0,0.0,0.0,1.348790e+09,1.379894e+09,1.363651e+09
699832,699832,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,64.0,1.0,0.0,1.0,3.0,0.0,10.0,1.349395e+09,1.381795e+09,1.355789e+09


Important parameters: 


*   early_stopping: tells the model after how many rounds to stop after it finds the smallest number of the evaluation metric
*   eval_metric: evaluation metric must be specified 

